# DenseNet Architecture 

In [1]:
from keras.layers import Input, Dense, Dropout, Activation, Concatenate, BatchNormalization
from keras.models import Model
from keras.layers import Conv2D, GlobalAveragePooling2D, AveragePooling2D
from keras.regularizers import l2
from keras.models import Model


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Transition Layer

In [2]:

def transition_layer(x, nb_channels):
    
 
    x = BatchNormalization(gamma_regularizer=l2(1e-4), beta_regularizer=l2(1e-4))(x)
    x = Activation('relu')(x)
    x = Conv2D(int(nb_channels*1), (1, 1), padding='same',
                      use_bias=False, kernel_regularizer=l2(1e-4))(x)
    

    x = AveragePooling2D((2, 2), strides=(2, 2))(x)
    return x


## Convolution Block

In [3]:
def convolution_block(x, nb_channels):
    # Standard (BN-ReLU-Conv)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_channels, (3, 3), padding='same')(x)
    

    return x


## Dense Block

In [4]:
def dense_block(x, nb_layers, nb_channels, growth_rate):
    
    x_list = [x]
    for i in range(nb_layers):
        cb = convolution_block(x, nb_channels)
        x_list.append(cb)
        x = Concatenate(axis=-1)(x_list)
        nb_channels += growth_rate
    return x, nb_channels


## Arch.

In [5]:
def DenseNet(input_shape=None, dense_blocks=4,  growth_rate=32, nb_classes=None, 
               weight_decay=1e-4, depth=40):

    nb_layers = [6,12,24,16] # For DenseNet-121

    
        
    img_input = Input(shape=input_shape)
    nb_channels = growth_rate * 2
    # Initial convolution layer
    x = Conv2D(nb_channels, (3,3), padding='same',strides=(1,1),
                      use_bias=False, kernel_regularizer=l2(weight_decay))(img_input)
    # Building dense blocks
    for block in range(dense_blocks):
        
        # Add dense block
        x, nb_channels = dense_block(x, nb_layers[block], nb_channels, growth_rate)
    
        if block < dense_blocks - 1:  
            x = transition_layer(x, nb_channels)
    
    x = BatchNormalization(gamma_regularizer=l2(weight_decay), beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)
    x = Model(img_input, x)

    x.summary()  



In [6]:
DenseNet(input_shape=(224,244,3), dense_blocks=4,  growth_rate=32, nb_classes=2,
               weight_decay=1e-4, depth=40)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 244, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 244, 64) 1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 224, 244, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 224, 244, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (